In [4]:
import torch
from generator import RandomGraphDataset
from network import Network

2024-03-20 15:10:27.214929: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/hadoop/lib/native
2024-03-20 15:10:27.810633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/hadoop/lib/native
2024-03-20 15:10:27.810704: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/hadoop/lib/native
2024-03-20 15:10:27.810709: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If 

In [16]:
n = 1000
p = 0.3
dataset_test_1000 = RandomGraphDataset(root='./data/OOD_1000', gen_num_graph=300, n=n, p=p)

Using 64 CPU cores
Generating 300 graphs


100%|██████████| 300/300 [03:19<00:00,  1.51it/s]
Processing...
 19%|█▊        | 56/300 [01:20<05:40,  1.40s/it]

In [ ]:
n = [20, 100]
p = 0.3
dataset_test_1000 = RandomGraphDataset(root='./data/train_500', gen_num_graph=300, n=n, p=p)

In [4]:
def train(model, train_dataset, validation_dataset=None, optimizer=None, epochs=10, batch_size=5):
    x_loss_weight = 0.5
    h_loss_weight = 1 - x_loss_weight

    loss_edges_train, loss_reach_train, loss_parents_train = [], [], []
    loss_edges_val, loss_reach_val, loss_parents_val = [], [], []

    for epoch in range(epochs):
        batch_count = len(train_dataset) // batch_size
        
        cumulated_loss_edges_epoch, cumulated_loss_reach_epoch, cumulated_loss_parents_epoch = 0, 0, 0

        for i in range(batch_count):
            model.train()
            cumulated_loss_edges, cumulated_loss_reach, cumulated_loss_parents = 0, 0, 0
            for j in range(i*batch_size, (i+1)*batch_size):
                graph = train_dataset[j] 
                loss_edges, loss_reach, loss_parents = model(graph)
                loss_edges_output, loss_edges_hints = loss_edges[0], loss_edges[1] # loss for the edges

                cumulated_loss_edges += x_loss_weight * loss_edges_output + h_loss_weight * loss_edges_hints
                cumulated_loss_reach += loss_reach
                cumulated_loss_parents += loss_parents

            cumulated_loss_edges /= batch_size
            cumulated_loss_reach /= batch_size
            cumulated_loss_parents /= batch_size
            
            optimizer.zero_grad()
            cumulated_loss_edges.backward()
            optimizer.step()

            cumulated_loss_edges_epoch += cumulated_loss_edges
            cumulated_loss_reach_epoch += cumulated_loss_reach
            cumulated_loss_parents_epoch += cumulated_loss_parents

        # Convert tensors to lists and append to the respective lists
        loss_edges_train.append(cumulated_loss_edges_epoch.item() / batch_count)
        loss_reach_train.append(cumulated_loss_reach_epoch.item() / batch_count)
        loss_parents_train.append(cumulated_loss_parents_epoch.item() / batch_count)

        if validation_dataset:
            model.eval()
            with torch.no_grad():
                cumulated_loss_edges_val, cumulated_loss_reach_val, cumulated_loss_parents_val = 0, 0, 0
                for k in range(len(validation_dataset)):
                    graph = validation_dataset[k]
                    loss_edges, loss_reach, loss_parents = model(graph)
                    loss_edges_output, loss_edges_hints = loss_edges[0], loss_edges[1] # loss for the edges

                    cumulated_loss_edges_val += x_loss_weight * loss_edges_output + h_loss_weight * loss_edges_hints
                    cumulated_loss_reach_val += loss_reach
                    cumulated_loss_parents_val += loss_parents 

                cumulated_loss_edges_val /= len(validation_dataset)
                cumulated_loss_reach_val /= len(validation_dataset)
                cumulated_loss_parents_val /= len(validation_dataset)

                loss_edges_val.append(cumulated_loss_edges_val.item())
                loss_reach_val.append(cumulated_loss_reach_val.item())
                loss_parents_val.append(cumulated_loss_parents_val.item())

                print(f'Epoch {epoch}, loss_edges {cumulated_loss_edges_epoch.item() / batch_count}, loss_reach {cumulated_loss_reach_epoch.item() / batch_count}, loss_parents {cumulated_loss_parents_epoch.item() / batch_count}, loss_edges_val {cumulated_loss_edges_val.item()}, loss_reach_val {cumulated_loss_reach_val.item()}, loss_parents_val {cumulated_loss_parents_val.item()}')
        
        else:
            print(f'Epoch {epoch}, loss_edges {cumulated_loss_edges_epoch.item() / batch_count}, loss_reach {cumulated_loss_reach_epoch.item() / batch_count}, loss_parents {cumulated_loss_parents_epoch.item() / batch_count}')

    if validation_dataset:
        return loss_edges_train, loss_reach_train, loss_parents_train, loss_edges_val, loss_reach_val, loss_parents_val
    return loss_edges_train, loss_reach_train, loss_parents_train

In [5]:
from torch.utils.data import random_split
n=[20, 100]
p=0.3
dataset = RandomGraphDataset(root='./data/medium', gen_num_graph=400, n=n, p=p)
train_dataset, test_dataset = random_split(dataset, [350, 50])

In [11]:
train_dataset, test_dataset = random_split(dataset, [350, 50])

In [12]:
optimizer = torch.optim.Adam
model = Network()
lr = 0.0002

loss_edges_train, loss_reach_train, loss_parents_train, loss_edges_val, loss_reach_val, loss_parents_val = train(model=model, train_dataset=train_dataset, validation_dataset=test_dataset,
      optimizer=optimizer(model.parameters(), lr=lr), epochs=120, batch_size=32)

Epoch 0, loss_edges 0.9657151222229003, loss_reach 9.6875, loss_parents 0.8648370742797852, loss_edges_val 0.8350149393081665, loss_reach_val 92.0, loss_parents_val 0.7820606827735901
Epoch 1, loss_edges 0.7250656604766845, loss_reach 99.0625, loss_parents 0.7639825820922852, loss_edges_val 0.5901945233345032, loss_reach_val 100.0, loss_parents_val 0.8380494117736816
Epoch 2, loss_edges 0.5594544887542725, loss_reach 100.0, loss_parents 0.8555074691772461, loss_edges_val 0.5696196556091309, loss_reach_val 100.0, loss_parents_val 0.8208716511726379
Epoch 3, loss_edges 0.5335549354553223, loss_reach 100.0, loss_parents 0.7794113636016846, loss_edges_val 0.5181726813316345, loss_reach_val 100.0, loss_parents_val 0.7190418243408203
Epoch 4, loss_edges 0.5030217170715332, loss_reach 100.0, loss_parents 0.6624913692474366, loss_edges_val 0.49758461117744446, loss_reach_val 100.0, loss_parents_val 0.6221373677253723
Epoch 5, loss_edges 0.4785293102264404, loss_reach 100.0, loss_parents 0.5897

In [2]:
import torch
torch.save(model.state_dict(), 'model_0002_350_120.pth')

NameError: name 'model' is not defined

In [5]:
model = Network()
model.load_state_dict(torch.load('model_0002_350_120.pth'))

<All keys matched successfully>

In [6]:
from generator import RandomGraphDataset

In [ ]:
# split the dataset into training and testing using the train_test_split function
from torch.utils.data import random_split
n=[200, 500]
p=0.3
dataset_test = RandomGraphDataset(root='./data/OOD/200', gen_num_graph=34, n=n, p=p)

In [11]:
x_loss_weight = h_loss_weight = 0.5
model.eval() 
test_dataset = dataset_test_1000
with torch.no_grad():
    cumulated_loss_edges_test, cumulated_loss_reach_test, cumulated_loss_parents_test = 0, 0, 0
    for graph in test_dataset:
        loss_edges, loss_reach, loss_parents = model(graph)
        loss_edges_output, loss_edges_hints = loss_edges[0], loss_edges[1]  # loss for the edges

        cumulated_loss_edges_test += x_loss_weight * loss_edges_output + h_loss_weight * loss_edges_hints
        cumulated_loss_reach_test += loss_reach
        cumulated_loss_parents_test += loss_parents

    cumulated_loss_edges_test /= len(test_dataset)
    cumulated_loss_reach_test /= len(test_dataset)
    cumulated_loss_parents_test /= len(test_dataset)

In [12]:
cumulated_loss_edges_test, cumulated_loss_reach_test, cumulated_loss_parents_test

(tensor(0.3119), tensor(0.), tensor(0.0014))